In [137]:
#lbbVIXAnalysis-6-1
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',400)
pd.set_option('display.max_rows',400)
pd.options.display.float_format = '{:,.2f}'.format
## or with magic %precision %.2f

import datetime as dt

import os
os.getcwd()


'c:\\users\\cinderella\\datasci\\datascience\\elitedatascience\\07-cornerstoneproject\\Final'

In [138]:
os.chdir('c:\\users\\cinderella\\datasci\\datascience\\elitedatascience\\07-cornerstoneproject\\Final')

##  Get the slope created in columns so that we can add the slope metrics

In the last section VIXAnalysis-5, several uncovered issues came up. Any values that were in the hundreds were investigated, and dealt with. We also added new features that will further help us in defining the attributes of the vix term slope that are the most indicative of market movement. 

I was hopeful that plotting the metrics would help, but that only served as a diversion to getting the relevant prices associated with term positions. 

I'm still not sure about why there are such gaps in the historical data in regards to the term structure. Observations from historical vix at vixcentral.com shows that there do appear to be gaps, but why? Also, on the CBOE (was CFE where I researched back in the day)..they list what they define as "term structure" which has later month contracts that I cannot find anywhere on the site. 

The goal for this workbook is to get prices for each term by date and compute the slope metrics we need:
More pre-processing - 
    fixed more data problems 1 - 4
    deleted column 0 and 10 - 12 of VXTerms
    trimmed data file and saved before pivoting
    two pivots, one a multi key and the other only vxTerms for Settle price

Slope length from first term to last
Slope min and max and the difference in range between HL and OC for EACH term
nTerm and fTerm values and curve original style
nTerm2 and fTerm2 and curve2 for the future futures - original style
nTerm2b and fTerm2b and curve2b future futures - actual curve * 100
nterm3 and fTerm3 and curve3 actual curve * 100
two new signal features, 1st over 2nd term and 8th over 9th term




In [139]:
df=pd.read_csv('df-5F.csv', index_col=0)

In [140]:
df['tradeDate'] = pd.to_datetime(df['tradeDate'])
df['frontMthXP'] = pd.to_datetime(df['frontMthXP'])
df['LTD'] = pd.to_datetime(df['LTD'])
df['diffDates'] = df['LTD'] - df['tradeDate']
df['9MthsStart'] = pd.to_datetime(df['9MthsStart'])
df['7MthsStart'] = pd.to_datetime(df['7MthsStart'])

In [141]:
df.loc[(df.tradeDate > '08/10/2020')]

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
33599,2020-08-11,8 days,2020,Q (Aug 2020),8,1,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,23.81,25.40,22.65,24.68,24.93,0.25,2.75,0.87,8,9,10,11,12,1,2,3,4,5,6,7
33600,2020-08-11,8 days,2020,U (Sep 2020),9,2,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.20,28.35,26.19,27.90,28.18,0.28,2.16,0.70,8,9,10,11,12,1,2,3,4,5,6,7
33601,2020-08-11,8 days,2020,V (Oct 2020),10,3,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,30.05,31.07,29.20,30.71,30.98,0.27,1.87,0.66,8,9,10,11,12,1,2,3,4,5,6,7
33602,2020-08-11,8 days,2020,X (Nov 2020),11,4,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,28.60,29.35,27.85,29.02,29.27,0.25,1.50,0.42,8,9,10,11,12,1,2,3,4,5,6,7
33603,2020-08-11,8 days,2020,Z (Dec 2020),12,5,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.65,28.33,27.05,28.05,28.23,0.18,1.28,0.40,8,9,10,11,12,1,2,3,4,5,6,7
33604,2020-08-11,8 days,2021,F (Jan 2021),1,6,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,28.15,28.67,27.52,28.45,28.62,0.18,1.15,0.30,8,9,10,11,12,1,2,3,4,5,6,7
33605,2020-08-11,8 days,2021,G (Feb 2021),2,7,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.77,28.24,27.15,28.04,28.23,0.19,1.09,0.27,8,9,10,11,12,1,2,3,4,5,6,7
33606,2020-08-11,8 days,2021,H (Mar 2021),3,8,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.00,28.00,26.95,27.95,27.98,0.03,1.05,0.95,8,9,10,11,12,1,2,3,4,5,6,7
33607,2020-08-11,8 days,2021,J (Apr 2021),4,9,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,0.00,27.25,27.70,0.00,27.00,27.00,-0.45,0.00,8,9,10,11,12,1,2,3,4,5,6,7
33608,2020-08-12,7 days,2020,Q (Aug 2020),8,1,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,24.64,25.03,23.03,23.35,23.27,-0.08,2.00,-1.29,8,9,10,11,12,1,2,3,4,5,6,7


In [142]:
df.loc[(df.Futures == 'H (Mar 2018)')]

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
26495,2017-06-26,-5 days,2018,H (Mar 2018),3,9,2017,2017-06-21,2017-06-21,2016-09-24,2016-11-23,0.00,0.00,0.00,0.00,16.68,16.68,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6
26504,2017-06-27,-6 days,2018,H (Mar 2018),3,9,2017,2017-06-21,2017-06-21,2016-09-24,2016-11-23,16.98,17.15,16.95,17.15,17.12,-0.02,0.20,0.17,7,8,9,10,11,12,1,2,3,4,5,6
26513,2017-06-28,-7 days,2018,H (Mar 2018),3,9,2017,2017-06-21,2017-06-21,2016-09-24,2016-11-23,17.15,17.15,16.90,17.00,17.00,0.00,0.25,-0.15,7,8,9,10,11,12,1,2,3,4,5,6
26522,2017-06-29,-8 days,2018,H (Mar 2018),3,9,2017,2017-06-21,2017-06-21,2016-09-24,2016-11-23,16.90,17.55,16.90,17.18,17.15,-0.03,0.65,0.28,7,8,9,10,11,12,1,2,3,4,5,6
26531,2017-06-30,-9 days,2018,H (Mar 2018),3,9,2017,2017-06-21,2017-06-21,2016-09-24,2016-11-23,17.15,17.22,17.05,17.20,17.20,0.00,0.17,0.05,7,8,9,10,11,12,1,2,3,4,5,6
26540,2017-07-03,16 days,2018,H (Mar 2018),3,9,2017,2017-07-19,2017-07-19,2016-10-22,2016-12-21,17.12,17.35,16.95,17.34,17.30,-0.04,0.40,0.22,7,8,9,10,11,12,1,2,3,4,5,6
26549,2017-07-05,14 days,2018,H (Mar 2018),3,9,2017,2017-07-19,2017-07-19,2016-10-22,2016-12-21,17.17,17.36,17.10,17.28,17.25,-0.03,0.26,0.11,7,8,9,10,11,12,1,2,3,4,5,6
26558,2017-07-06,13 days,2018,H (Mar 2018),3,9,2017,2017-07-19,2017-07-19,2016-10-22,2016-12-21,17.20,17.46,17.04,17.46,17.40,-0.06,0.42,0.26,7,8,9,10,11,12,1,2,3,4,5,6
26567,2017-07-07,12 days,2018,H (Mar 2018),3,9,2017,2017-07-19,2017-07-19,2016-10-22,2016-12-21,17.35,17.45,17.30,17.40,17.40,0.00,0.15,0.05,7,8,9,10,11,12,1,2,3,4,5,6
26576,2017-07-10,9 days,2018,H (Mar 2018),3,9,2017,2017-07-19,2017-07-19,2016-10-22,2016-12-21,17.30,17.36,17.15,17.17,17.15,-0.02,0.21,-0.13,7,8,9,10,11,12,1,2,3,4,5,6


In [143]:
df.dtypes

tradeDate           datetime64[ns]
diffDates          timedelta64[ns]
futYear                      int64
Futures                     object
futMth                       int64
vxTerm                       int64
calYear                      int64
LTD                 datetime64[ns]
frontMthXP          datetime64[ns]
9MthsStart          datetime64[ns]
7MthsStart          datetime64[ns]
Open                       float64
High                       float64
Low                        float64
Close                      float64
Settle                     float64
diffSettleClose            float64
rangeDayHL                 float64
rangeDayOC                 float64
frontMthNum                  int64
numMthT02                    int64
numMthT03                    int64
numMthT04                    int64
numMthT05                    int64
numMthT06                    int64
numMthT07                    int64
numMthT08                    int64
numMthT09                    int64
numMthT10           

In [144]:
df.head(100)

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
0,2004-03-26,-10 days,2004,K (May 2004),5,2,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.24,21.25,20.27,20.27,20.32,0.05,0.98,-0.97,4,5,6,7,8,9,10,11,12,1,2,3
1,2004-03-26,-10 days,2004,M (Jun 2004),6,3,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.33,20.37,20.10,20.20,20.16,-0.04,0.27,-0.13,4,5,6,7,8,9,10,11,12,1,2,3
2,2004-03-26,-10 days,2004,Q (Aug 2004),8,5,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.04,20.13,20.00,20.06,20.11,0.05,0.13,0.02,4,5,6,7,8,9,10,11,12,1,2,3
3,2004-03-26,-10 days,2004,X (Nov 2004),11,8,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.17,21.37,20.88,20.89,20.94,0.05,0.49,-0.28,4,5,6,7,8,9,10,11,12,1,2,3
4,2004-03-29,-13 days,2004,K (May 2004),5,2,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.90,19.99,19.75,19.77,19.80,0.03,0.24,-0.13,4,5,6,7,8,9,10,11,12,1,2,3
5,2004-03-29,-13 days,2004,M (Jun 2004),6,3,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.85,19.85,19.73,19.76,19.77,0.01,0.12,-0.09,4,5,6,7,8,9,10,11,12,1,2,3
6,2004-03-29,-13 days,2004,Q (Aug 2004),8,5,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.76,19.76,19.76,19.76,19.68,-0.08,0.00,0.00,4,5,6,7,8,9,10,11,12,1,2,3
7,2004-03-29,-13 days,2004,X (Nov 2004),11,8,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.83,20.97,20.83,20.97,20.98,0.01,0.14,0.14,4,5,6,7,8,9,10,11,12,1,2,3
8,2004-03-30,-14 days,2004,K (May 2004),5,2,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.86,20.04,19.61,19.61,19.62,0.01,0.43,-0.25,4,5,6,7,8,9,10,11,12,1,2,3
9,2004-03-30,-14 days,2004,M (Jun 2004),6,3,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.85,19.88,19.73,19.73,19.79,0.06,0.15,-0.12,4,5,6,7,8,9,10,11,12,1,2,3


In [145]:
df.iloc[[6], :] #the row 6 above is not correct, let's isolate it to be sure our code works.

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
6,2004-03-29,-13 days,2004,Q (Aug 2004),8,5,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.76,19.76,19.76,19.76,19.68,-0.08,0.00,0.00,4,5,6,7,8,9,10,11,12,1,2,3


In [146]:
## delete the row we isolated above (this cell must stay in this location in the notebook,
## otherwise if it gets out of order it could demolish a different row on the 6th line.)
df.drop([df.index[6]])

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
0,2004-03-26,-10 days,2004,K (May 2004),5,2,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.24,21.25,20.27,20.27,20.32,0.05,0.98,-0.97,4,5,6,7,8,9,10,11,12,1,2,3
1,2004-03-26,-10 days,2004,M (Jun 2004),6,3,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.33,20.37,20.10,20.20,20.16,-0.04,0.27,-0.13,4,5,6,7,8,9,10,11,12,1,2,3
2,2004-03-26,-10 days,2004,Q (Aug 2004),8,5,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.04,20.13,20.00,20.06,20.11,0.05,0.13,0.02,4,5,6,7,8,9,10,11,12,1,2,3
3,2004-03-26,-10 days,2004,X (Nov 2004),11,8,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.17,21.37,20.88,20.89,20.94,0.05,0.49,-0.28,4,5,6,7,8,9,10,11,12,1,2,3
4,2004-03-29,-13 days,2004,K (May 2004),5,2,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.90,19.99,19.75,19.77,19.80,0.03,0.24,-0.13,4,5,6,7,8,9,10,11,12,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33673,2020-08-21,-2 days,2020,Z (Dec 2020),12,4,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.77,28.45,27.66,28.24,28.18,-0.06,0.79,0.47,9,10,11,12,1,2,3,4,5,6,7,8
33674,2020-08-21,-2 days,2021,F (Jan 2021),1,5,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.90,28.60,27.82,28.58,28.52,-0.05,0.78,0.68,9,10,11,12,1,2,3,4,5,6,7,8
33675,2020-08-21,-2 days,2021,G (Feb 2021),2,6,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.93,28.75,27.85,28.75,28.70,-0.05,0.90,0.82,9,10,11,12,1,2,3,4,5,6,7,8
33676,2020-08-21,-2 days,2021,H (Mar 2021),3,7,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.57,28.40,27.55,28.38,28.38,-0.00,0.85,0.81,9,10,11,12,1,2,3,4,5,6,7,8


In [147]:
df.vxTerm.unique()

array([ 2,  3,  5,  8,  1,  4,  7, 12,  6,  9,  0, 10, 11], dtype=int64)

In [148]:
#df[df.vxTerm.isin([1])]

In [149]:
df.sort_values(by=['tradeDate', 'vxTerm'])

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
0,2004-03-26,-10 days,2004,K (May 2004),5,2,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.24,21.25,20.27,20.27,20.32,0.05,0.98,-0.97,4,5,6,7,8,9,10,11,12,1,2,3
1,2004-03-26,-10 days,2004,M (Jun 2004),6,3,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.33,20.37,20.10,20.20,20.16,-0.04,0.27,-0.13,4,5,6,7,8,9,10,11,12,1,2,3
2,2004-03-26,-10 days,2004,Q (Aug 2004),8,5,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.04,20.13,20.00,20.06,20.11,0.05,0.13,0.02,4,5,6,7,8,9,10,11,12,1,2,3
3,2004-03-26,-10 days,2004,X (Nov 2004),11,8,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.17,21.37,20.88,20.89,20.94,0.05,0.49,-0.28,4,5,6,7,8,9,10,11,12,1,2,3
4,2004-03-29,-13 days,2004,K (May 2004),5,2,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.90,19.99,19.75,19.77,19.80,0.03,0.24,-0.13,4,5,6,7,8,9,10,11,12,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33673,2020-08-21,-2 days,2020,Z (Dec 2020),12,4,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.77,28.45,27.66,28.24,28.18,-0.06,0.79,0.47,9,10,11,12,1,2,3,4,5,6,7,8
33674,2020-08-21,-2 days,2021,F (Jan 2021),1,5,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.90,28.60,27.82,28.58,28.52,-0.05,0.78,0.68,9,10,11,12,1,2,3,4,5,6,7,8
33675,2020-08-21,-2 days,2021,G (Feb 2021),2,6,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.93,28.75,27.85,28.75,28.70,-0.05,0.90,0.82,9,10,11,12,1,2,3,4,5,6,7,8
33676,2020-08-21,-2 days,2021,H (Mar 2021),3,7,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.57,28.40,27.55,28.38,28.38,-0.00,0.85,0.81,9,10,11,12,1,2,3,4,5,6,7,8


Gross errors everywhere.
Why on the date 10/2/2006 do we have a futures contract for 2008 reporting?<br>
Because it was in the raw data for K(May 2008)..someone needs better QA, <br>
I'm beginning to think we cannot trust any information from 2004 and 2005 and not sure about 2006<br>
This is what FREE data means

In [150]:
df['problems'] = np.where((df.futYear == 2008) & (df.calYear == 2006), 'X', '')
df.loc[(df['problems'] == 'X')]

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems
1931,2006-04-21,-3 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,6,7,8,9,10,11,12,1,2,3,4,X
1938,2006-04-24,-6 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,0.00,0.00,0.00,0.00,16.62,16.62,0.00,0.00,5,6,7,8,9,10,11,12,1,2,3,4,X
1945,2006-04-25,-7 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,16.64,16.75,16.64,16.75,16.66,-0.09,0.11,0.11,5,6,7,8,9,10,11,12,1,2,3,4,X
1952,2006-04-26,-8 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,16.73,16.79,16.73,16.79,16.82,0.03,0.06,0.06,5,6,7,8,9,10,11,12,1,2,3,4,X
1959,2006-04-27,-9 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,16.79,16.93,16.79,16.93,16.89,-0.04,0.14,0.14,5,6,7,8,9,10,11,12,1,2,3,4,X
1966,2006-04-28,-10 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,16.97,17.03,16.97,17.03,16.98,-0.05,0.06,0.06,5,6,7,8,9,10,11,12,1,2,3,4,X
1973,2006-05-01,15 days,2008,K (May 2008),5,1,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,0.00,0.00,0.00,0.00,16.95,16.95,0.00,0.00,5,6,7,8,9,10,11,12,1,2,3,4,X
1980,2006-05-02,14 days,2008,K (May 2008),5,1,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,17.04,17.13,17.04,17.13,17.08,-0.05,0.09,0.09,5,6,7,8,9,10,11,12,1,2,3,4,X
1987,2006-05-03,13 days,2008,K (May 2008),5,1,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,17.18,17.18,17.18,17.18,17.12,-0.06,0.00,0.00,5,6,7,8,9,10,11,12,1,2,3,4,X
1994,2006-05-04,12 days,2008,K (May 2008),5,1,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,17.12,17.16,17.12,17.16,17.11,-0.05,0.04,0.04,5,6,7,8,9,10,11,12,1,2,3,4,X


In [151]:
fOldFuturesRemoved = pd.DataFrame() #create new df to hold problem futures

In [152]:
cond = df['problems'] == 'X'
rows = df.loc[cond, :]
fOldFuturesRemoved = fOldFuturesRemoved.append(rows, ignore_index=True)
fOldFuturesRemoved.tail(5) ## move the problem rows into the new database and confirm

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems
171,2006-12-22,-3 days,2008,K (May 2008),5,5,2006,2006-12-19,2006-12-20,2006-03-25,2006-05-24,0.00,0.00,0.00,0.00,16.39,16.39,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,X
172,2006-12-26,-7 days,2008,K (May 2008),5,5,2006,2006-12-19,2006-12-20,2006-03-25,2006-05-24,0.00,0.00,0.00,0.00,16.37,16.37,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,X
173,2006-12-27,-8 days,2008,K (May 2008),5,5,2006,2006-12-19,2006-12-20,2006-03-25,2006-05-24,0.00,0.00,0.00,0.00,16.40,16.40,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,X
174,2006-12-28,-9 days,2008,K (May 2008),5,5,2006,2006-12-19,2006-12-20,2006-03-25,2006-05-24,0.00,0.00,0.00,0.00,16.54,16.54,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,X
175,2006-12-29,-10 days,2008,K (May 2008),5,5,2006,2006-12-19,2006-12-20,2006-03-25,2006-05-24,0.00,0.00,0.00,0.00,16.49,16.49,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,X


In [153]:
## remove the rows from the original df
df.drop(rows.index, inplace=True)

In [154]:
df['problems2'] = np.where((df.futYear == 2008) & (df.futMth == 5) & (df.tradeDate < '2007-8-22'), 'X', '')
df.loc[(df['problems2'] == 'X')]

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems,problems2
3255,2007-01-03,13 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,0.00,0.00,0.00,0.00,16.49,16.49,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,X
3264,2007-01-04,12 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,0.00,0.00,0.00,0.00,16.47,16.47,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,X
3273,2007-01-05,11 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,16.58,16.58,16.58,16.58,16.54,-0.04,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,X
3282,2007-01-08,8 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,0.00,0.00,0.00,0.00,16.54,16.54,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,X
3291,2007-01-09,7 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,16.60,16.60,16.60,16.60,16.54,-0.06,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,X
3300,2007-01-10,6 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,0.00,0.00,0.00,0.00,16.54,16.54,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,X
3309,2007-01-11,5 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,16.40,16.59,16.40,16.59,16.54,-0.05,0.19,0.19,1,2,3,4,5,6,7,8,9,10,11,12,,X
3318,2007-01-12,4 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,16.64,16.64,16.64,16.64,16.54,-0.10,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,X
3327,2007-01-16,0 days,2008,K (May 2008),5,5,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,16.41,16.41,16.40,16.40,16.42,0.02,0.01,-0.01,1,2,3,4,5,6,7,8,9,10,11,12,,X
3336,2007-01-17,-1 days,2008,K (May 2008),5,4,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,16.32,16.55,16.32,16.55,16.46,-0.09,0.23,0.23,2,3,4,5,6,7,8,9,10,11,12,1,,X


In [155]:
cond2 = df['problems2'] == 'X'
rows2 = df.loc[cond2, :]
fOldFuturesRemoved = fOldFuturesRemoved.append(rows2, ignore_index=True)

In [156]:
df.drop(rows2.index, inplace=True)

Take a guess at when the beginning of the 9th month will be but be generous. When you see the 9th month beginning in the location below, make a note of the start of the rows that begin the 9th month, then you can change your query to be exact and include those - but get rid of the others (the above query was originally for'2006-11-22' but seeing that 11-15-2006 is the actual beginning of the 9th month I will change the query above)

In [157]:
df['problems3'] = np.where((df.futYear == 2007) & (df.futMth == 8) & (df.tradeDate < '2006-11-15'), 'X', '')
df.loc[(df['problems3'] == 'X')]

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems,problems2,problems3
2224,2006-06-21,-1 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,0.00,0.00,0.00,0.00,16.34,16.34,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2231,2006-06-22,-2 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,0.00,0.00,0.00,0.00,16.49,16.49,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2238,2006-06-23,-3 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,0.00,0.00,0.00,0.00,16.54,16.54,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2245,2006-06-26,-6 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,16.44,16.44,16.44,16.44,16.47,0.03,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2252,2006-06-27,-7 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,16.42,16.42,16.42,16.42,16.52,0.10,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2259,2006-06-28,-8 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,16.62,16.62,16.62,16.62,16.59,-0.03,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2266,2006-06-29,-9 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,16.41,16.45,16.36,16.36,16.35,-0.01,0.09,-0.05,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2273,2006-06-30,-10 days,2007,Q (Aug 2007),8,2,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,0.00,0.00,0.00,0.00,16.33,16.33,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2280,2006-07-03,15 days,2007,Q (Aug 2007),8,2,2006,2006-07-18,2006-07-19,2005-10-22,2005-12-21,0.00,0.00,0.00,0.00,16.33,16.33,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,X
2287,2006-07-05,13 days,2007,Q (Aug 2007),8,2,2006,2006-07-18,2006-07-19,2005-10-22,2005-12-21,16.21,16.40,15.47,16.40,16.33,-0.07,0.93,0.19,7,8,9,10,11,12,1,2,3,4,5,6,,,X


In [158]:
cond3 = df['problems3'] == 'X'
rows3 = df.loc[cond3, :]
fOldFuturesRemoved = fOldFuturesRemoved.append(rows3, ignore_index=True)
fOldFuturesRemoved

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems,problems2,problems3
0,2006-04-21,-3 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,6,7,8,9,10,11,12,1,2,3,4,X,NaN,NaN
1,2006-04-24,-6 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,0.00,0.00,0.00,0.00,16.62,16.62,0.00,0.00,5,6,7,8,9,10,11,12,1,2,3,4,X,NaN,NaN
2,2006-04-25,-7 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,16.64,16.75,16.64,16.75,16.66,-0.09,0.11,0.11,5,6,7,8,9,10,11,12,1,2,3,4,X,NaN,NaN
3,2006-04-26,-8 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,16.73,16.79,16.73,16.79,16.82,0.03,0.06,0.06,5,6,7,8,9,10,11,12,1,2,3,4,X,NaN,NaN
4,2006-04-27,-9 days,2008,K (May 2008),5,1,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,16.79,16.93,16.79,16.93,16.89,-0.04,0.14,0.14,5,6,7,8,9,10,11,12,1,2,3,4,X,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,2006-11-08,6 days,2007,Q (Aug 2007),8,10,2006,2006-11-14,2006-11-15,2006-02-18,2006-04-19,0.00,0.00,0.00,0.00,15.53,15.53,0.00,0.00,11,12,1,2,3,4,5,6,7,8,9,10,,,X
435,2006-11-09,5 days,2007,Q (Aug 2007),8,10,2006,2006-11-14,2006-11-15,2006-02-18,2006-04-19,15.50,15.50,15.45,15.45,15.53,0.08,0.05,-0.05,11,12,1,2,3,4,5,6,7,8,9,10,,,X
436,2006-11-10,4 days,2007,Q (Aug 2007),8,10,2006,2006-11-14,2006-11-15,2006-02-18,2006-04-19,0.00,0.00,0.00,0.00,15.57,15.57,0.00,0.00,11,12,1,2,3,4,5,6,7,8,9,10,,,X
437,2006-11-13,1 days,2007,Q (Aug 2007),8,10,2006,2006-11-14,2006-11-15,2006-02-18,2006-04-19,15.55,15.55,15.50,15.50,15.57,0.07,0.05,-0.05,11,12,1,2,3,4,5,6,7,8,9,10,,,X


In [159]:
df.drop(rows3.index, inplace=True)

In [160]:
df.loc[(df['vxTerm'] == 12)]

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems,problems2,problems3
148,2004-05-19,-1 days,2004,K (May 2004),5,12,2004,2004-05-18,2004-05-19,2003-08-23,2003-10-22,0.00,0.00,0.00,0.00,18.36,18.36,0.00,0.00,6,7,8,9,10,11,12,1,2,3,4,5,,,
219,2004-06-16,-1 days,2004,M (Jun 2004),6,12,2004,2004-06-15,2004-06-16,2003-09-20,2003-11-19,0.00,0.00,0.00,0.00,14.00,14.00,0.00,0.00,7,8,9,10,11,12,1,2,3,4,5,6,,,
391,2004-08-18,-1 days,2004,Q (Aug 2004),8,12,2004,2004-08-17,2004-08-18,2003-11-22,2004-01-21,0.00,0.00,0.00,0.00,17.49,17.49,0.00,0.00,9,10,11,12,1,2,3,4,5,6,7,8,,,
465,2004-09-15,-1 days,2004,U (Sep 2004),9,12,2004,2004-09-14,2004-09-15,2003-12-20,2004-02-18,0.00,0.00,0.00,0.00,13.72,13.72,0.00,0.00,10,11,12,1,2,3,4,5,6,7,8,9,,,
625,2004-11-17,-1 days,2004,X (Nov 2004),11,12,2004,2004-11-16,2004-11-17,2004-02-21,2004-04-21,0.00,0.00,0.00,0.00,12.84,12.84,0.00,0.00,12,1,2,3,4,5,6,7,8,9,10,11,,,
1030,2005-05-18,-1 days,2005,K (May 2005),5,12,2005,2005-05-17,2005-05-18,2004-08-21,2004-10-20,0.00,0.00,0.00,0.00,13.86,13.86,0.00,0.00,6,7,8,9,10,11,12,1,2,3,4,5,,,
1217,2005-08-17,-1 days,2005,Q (Aug 2005),8,12,2005,2005-08-16,2005-08-17,2004-11-20,2005-01-19,0.00,0.00,0.00,0.00,12.82,12.82,0.00,0.00,9,10,11,12,1,2,3,4,5,6,7,8,,,
1391,2005-10-19,-1 days,2005,V (Oct 2005),10,12,2005,2005-10-18,2005-10-19,2005-01-22,2005-03-23,0.00,0.00,0.00,0.00,15.17,15.17,0.00,0.00,11,12,1,2,3,4,5,6,7,8,9,10,,,
1470,2005-11-16,-1 days,2005,X (Nov 2005),11,12,2005,2005-11-15,2005-11-16,2005-02-19,2005-04-20,0.00,0.00,0.00,0.00,12.31,12.31,0.00,0.00,12,1,2,3,4,5,6,7,8,9,10,11,,,
1565,2005-12-21,-1 days,2005,Z (Dec 2005),12,12,2005,2005-12-20,2005-12-21,2005-03-26,2005-05-25,0.00,0.00,0.00,0.00,10.18,10.18,0.00,0.00,1,2,3,4,5,6,7,8,9,10,11,12,,,


It is possible that we have some data in the vxTerm 12 due to the front month coming in at the same day the old term is expiring. In general, the expiring month year should equal the calendar year. In the above example, there is no reason May 2007 futures should show a month 12 in the previous year (2006, item 12 above). On line 14 above, there is reason to keep the Jun 2006 listing for 6/21/2006 because that is the settlement date for June. So yet another problem rule. This one will remove any row that has vxTerm 12 and futYear > calYear

In [161]:
df['problems4'] = np.where((df.futYear > df.calYear) & (df.vxTerm == 12), 'X', '')
df.loc[(df['problems4'] == 'X')]

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems,problems2,problems3,problems4
2056,2006-05-17,-1 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,0.00,0.00,0.00,0.00,16.09,16.09,0.00,0.00,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2062,2006-05-18,-2 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,16.00,16.00,15.95,15.95,16.01,0.06,0.05,-0.05,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2069,2006-05-19,-3 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,15.99,15.99,15.87,15.87,15.93,0.06,0.12,-0.12,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2076,2006-05-22,-6 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,15.87,16.11,15.87,16.11,16.06,-0.05,0.24,0.24,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2083,2006-05-23,-7 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,15.43,15.92,15.43,15.92,15.89,-0.03,0.49,0.49,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2090,2006-05-24,-8 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,15.82,16.05,15.82,15.95,16.06,0.11,0.23,0.13,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2097,2006-05-25,-9 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,15.98,16.14,15.97,16.08,16.05,-0.03,0.17,0.10,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2104,2006-05-26,-10 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,15.97,16.10,15.95,15.95,16.03,0.08,0.15,-0.02,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2111,2006-05-30,-14 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,16.06,16.31,16.06,16.31,16.26,-0.05,0.25,0.25,6,7,8,9,10,11,12,1,2,3,4,5,,,,X
2118,2006-05-31,-15 days,2007,K (May 2007),5,12,2006,2006-05-16,2006-05-17,2005-08-20,2005-10-19,0.00,0.00,0.00,0.00,16.39,16.39,0.00,0.00,6,7,8,9,10,11,12,1,2,3,4,5,,,,X


In [162]:
cond4 = df['problems4'] == 'X'
rows4 = df.loc[cond4, :]
fOldFuturesRemoved = fOldFuturesRemoved.append(rows4, ignore_index=True)
fOldFuturesRemoved.tail(5)

,tradeDate,diffDates,futYear,Futures,futMth,vxTerm,calYear,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,problems,problems2,problems3,problems4
546,2019-12-12,6 days,2020,X (Nov 2020),11,12,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,19.00,19.20,19.00,19.20,19.12,-0.07,0.20,0.20,12,1,2,3,4,5,6,7,8,9,10,11,,,,X
547,2019-12-13,5 days,2020,X (Nov 2020),11,12,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,19.40,19.40,19.07,19.13,18.98,-0.15,0.33,-0.27,12,1,2,3,4,5,6,7,8,9,10,11,,,,X
548,2019-12-16,2 days,2020,X (Nov 2020),11,12,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,0.00,18.65,18.95,0.00,18.68,18.68,-0.30,0.00,12,1,2,3,4,5,6,7,8,9,10,11,,,,X
549,2019-12-17,1 days,2020,X (Nov 2020),11,12,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,18.99,19.07,18.80,18.90,18.82,-0.07,0.27,-0.09,12,1,2,3,4,5,6,7,8,9,10,11,,,,X
550,2019-12-18,0 days,2020,X (Nov 2020),11,12,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,18.95,19.15,18.79,18.81,18.75,-0.06,0.36,-0.14,12,1,2,3,4,5,6,7,8,9,10,11,,,,X


In [163]:
df.drop(rows4.index, inplace=True)

In [164]:
df.groupby(['tradeDate','vxTerm']).size()

tradeDate   vxTerm
2004-03-26  2         1
            3         1
            5         1
            8         1
2004-03-29  2         1
                     ..
2020-08-21  4         1
            5         1
            6         1
            7         1
            8         1
Length: 33087, dtype: int64

In [165]:
#Do we need any values for vxTerm 10? Are they for real?
#df['problems5'] = np.where((df.futYear > df.calYear) & (df.vxTerm == 10), 'X', '')
#df.loc[(df['problems5'] == 'X')]


In [166]:
#cond5 = df['problems5'] == 'X'
#rows5 = df.loc[cond5, :]
#fOldFuturesRemoved = fOldFuturesRemoved.append(rows4, ignore_index=True)
#fOldFuturesRemoved.tail(5)

In [167]:
#df.drop(rows5.index, inplace=True)

In [168]:
df.groupby(['tradeDate','vxTerm']).size()

tradeDate   vxTerm
2004-03-26  2         1
            3         1
            5         1
            8         1
2004-03-29  2         1
                     ..
2020-08-21  4         1
            5         1
            6         1
            7         1
            8         1
Length: 33087, dtype: int64

In [169]:
df.dtypes

tradeDate           datetime64[ns]
diffDates          timedelta64[ns]
futYear                      int64
Futures                     object
futMth                       int64
vxTerm                       int64
calYear                      int64
LTD                 datetime64[ns]
frontMthXP          datetime64[ns]
9MthsStart          datetime64[ns]
7MthsStart          datetime64[ns]
Open                       float64
High                       float64
Low                        float64
Close                      float64
Settle                     float64
diffSettleClose            float64
rangeDayHL                 float64
rangeDayOC                 float64
frontMthNum                  int64
numMthT02                    int64
numMthT03                    int64
numMthT04                    int64
numMthT05                    int64
numMthT06                    int64
numMthT07                    int64
numMthT08                    int64
numMthT09                    int64
numMthT10           

In [170]:
df.drop(['problems','problems2','problems3','problems4'], axis=1, inplace=True)

In [171]:
df.to_csv('df-6-0.csv')##cleaner, new features, nearly ready for pivot

In [172]:
dfVXTermMthNums = df.copy()

In [173]:
dfVXTermMthNums.dtypes

tradeDate           datetime64[ns]
diffDates          timedelta64[ns]
futYear                      int64
Futures                     object
futMth                       int64
vxTerm                       int64
calYear                      int64
LTD                 datetime64[ns]
frontMthXP          datetime64[ns]
9MthsStart          datetime64[ns]
7MthsStart          datetime64[ns]
Open                       float64
High                       float64
Low                        float64
Close                      float64
Settle                     float64
diffSettleClose            float64
rangeDayHL                 float64
rangeDayOC                 float64
frontMthNum                  int64
numMthT02                    int64
numMthT03                    int64
numMthT04                    int64
numMthT05                    int64
numMthT06                    int64
numMthT07                    int64
numMthT08                    int64
numMthT09                    int64
numMthT10           

In [174]:
dfVXTermMthNums.drop(['diffDates','futYear','Futures','futMth','vxTerm','calYear','LTD','Open','High','Low','Close','Settle','diffSettleClose','rangeDayHL','rangeDayOC'], 
                     axis=1, inplace=True)

In [175]:
dfVXTermMthNums


,tradeDate,frontMthXP,9MthsStart,7MthsStart,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
0,2004-03-26,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
1,2004-03-26,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
2,2004-03-26,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
3,2004-03-26,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
4,2004-03-29,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33673,2020-08-21,2020-08-19,2019-11-23,2020-01-22,9,10,11,12,1,2,3,4,5,6,7,8
33674,2020-08-21,2020-08-19,2019-11-23,2020-01-22,9,10,11,12,1,2,3,4,5,6,7,8
33675,2020-08-21,2020-08-19,2019-11-23,2020-01-22,9,10,11,12,1,2,3,4,5,6,7,8
33676,2020-08-21,2020-08-19,2019-11-23,2020-01-22,9,10,11,12,1,2,3,4,5,6,7,8


In [176]:
dfGStartEndDates = dfVXTermMthNums.groupby(['tradeDate', 'frontMthXP','9MthsStart','7MthsStart']).mean()

In [177]:
dfGStartEndDates.head()

,,,,frontMthNum,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12
tradeDate,frontMthXP,9MthsStart,7MthsStart,,,,,,,,,,,,
2004-03-26,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
2004-03-29,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
2004-03-30,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
2004-03-31,2004-03-17,2003-06-21,2003-08-20,4,5,6,7,8,9,10,11,12,1,2,3
2004-04-01,2004-04-21,2003-07-26,2003-09-24,4,5,6,7,8,9,10,11,12,1,2,3


In [178]:
dfGStartEndDates.to_csv('df-6-G1.csv')

In [179]:
## Now to remove the numMthTXX from the main df
df.dtypes


tradeDate           datetime64[ns]
diffDates          timedelta64[ns]
futYear                      int64
Futures                     object
futMth                       int64
vxTerm                       int64
calYear                      int64
LTD                 datetime64[ns]
frontMthXP          datetime64[ns]
9MthsStart          datetime64[ns]
7MthsStart          datetime64[ns]
Open                       float64
High                       float64
Low                        float64
Close                      float64
Settle                     float64
diffSettleClose            float64
rangeDayHL                 float64
rangeDayOC                 float64
frontMthNum                  int64
numMthT02                    int64
numMthT03                    int64
numMthT04                    int64
numMthT05                    int64
numMthT06                    int64
numMthT07                    int64
numMthT08                    int64
numMthT09                    int64
numMthT10           

In [180]:
df.drop(['numMthT02','numMthT03','numMthT04','numMthT05','numMthT06','numMthT07',
         'numMthT08','numMthT09','numMthT10','numMthT11','numMthT12'], 
                     axis=1, inplace=True)

In [181]:
df.dtypes

tradeDate           datetime64[ns]
diffDates          timedelta64[ns]
futYear                      int64
Futures                     object
futMth                       int64
vxTerm                       int64
calYear                      int64
LTD                 datetime64[ns]
frontMthXP          datetime64[ns]
9MthsStart          datetime64[ns]
7MthsStart          datetime64[ns]
Open                       float64
High                       float64
Low                        float64
Close                      float64
Settle                     float64
diffSettleClose            float64
rangeDayHL                 float64
rangeDayOC                 float64
frontMthNum                  int64
dtype: object

In [182]:
df = df[['tradeDate','diffDates','Futures','vxTerm','frontMthNum','LTD','frontMthXP','9MthsStart','7MthsStart',
         'Open','High','Low','Close','Settle','diffSettleClose','rangeDayHL','rangeDayOC',
         'futYear','futMth','calYear']]

In [183]:
df.head()


,tradeDate,diffDates,Futures,vxTerm,frontMthNum,LTD,frontMthXP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,diffSettleClose,rangeDayHL,rangeDayOC,futYear,futMth,calYear
0,2004-03-26,-10 days,K (May 2004),2,4,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.24,21.25,20.27,20.27,20.32,0.05,0.98,-0.97,2004,5,2004
1,2004-03-26,-10 days,M (Jun 2004),3,4,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.33,20.37,20.10,20.20,20.16,-0.04,0.27,-0.13,2004,6,2004
2,2004-03-26,-10 days,Q (Aug 2004),5,4,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.04,20.13,20.00,20.06,20.11,0.05,0.13,0.02,2004,8,2004
3,2004-03-26,-10 days,X (Nov 2004),8,4,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.17,21.37,20.88,20.89,20.94,0.05,0.49,-0.28,2004,11,2004
4,2004-03-29,-13 days,K (May 2004),2,4,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.90,19.99,19.75,19.77,19.80,0.03,0.24,-0.13,2004,5,2004


In [184]:
df.dtypes

tradeDate           datetime64[ns]
diffDates          timedelta64[ns]
Futures                     object
vxTerm                       int64
frontMthNum                  int64
LTD                 datetime64[ns]
frontMthXP          datetime64[ns]
9MthsStart          datetime64[ns]
7MthsStart          datetime64[ns]
Open                       float64
High                       float64
Low                        float64
Close                      float64
Settle                     float64
diffSettleClose            float64
rangeDayHL                 float64
rangeDayOC                 float64
futYear                      int64
futMth                       int64
calYear                      int64
dtype: object

In [185]:
df.to_csv('df-6-1.csv') #before pivot removed columns, reordered

In [186]:
# pivoting continues to give the error that ValueError: Index contains duplicate entries, cannot reshape

In [187]:
dfGBVT3 = df.groupby(['tradeDate','vxTerm'])[['Settle', 'diffSettleClose','rangeDayHL','rangeDayOC']].mean()

In [188]:
dfGBVT3.to_csv('df-6-G2.csv')
dfGBVT3.head()


Settle  diffSettleClose  rangeDayHL  rangeDayOC
tradeDate  vxTerm                                                 
2004-03-26 2        20.32             0.05        0.98       -0.97
           3        20.16            -0.04        0.27       -0.13
           5        20.11             0.05        0.13        0.02
           8        20.94             0.05        0.49       -0.28
2004-03-29 2        19.80             0.03        0.24       -0.13

In [189]:
df06A = pd.read_csv('df-6-G2.csv')

In [190]:
df06AP2 = df06A.pivot(index = 'tradeDate', columns = 'vxTerm', values = ['Settle','diffSettleClose',
                      'rangeDayHL','rangeDayOC'])

In [191]:
df06AP2.tail()
df06AP2.reset_index()
df06AP2.tail(100)

Settle                                                            \
vxTerm         0     1     2     3     4     5     6     7     8     9   10   
tradeDate                                                                     
2020-04-01    nan 51.33 44.62 40.38 36.77 34.38 32.52 31.95 30.75 32.65 nan   
2020-04-02    nan 47.17 42.02 38.35 35.42 33.42 31.68 31.23 30.10 31.32 nan   
2020-04-03    nan 44.92 40.90 37.92 35.02 33.08 31.50 31.27 29.98 32.08 nan   
2020-04-06    nan 41.77 37.92 35.88 33.48 31.73 30.32 30.27 28.93 29.00 nan   
2020-04-07    nan 44.02 39.67 36.67 34.10 32.52 31.65 31.38 29.90 28.30 nan   
2020-04-08    nan 42.38 38.30 36.08 33.55 31.98 31.10 30.93 29.32 26.98 nan   
2020-04-09    nan 41.27 36.92 34.65 32.52 31.15 30.38 30.32 28.77 28.68 nan   
2020-04-13    nan 40.58 36.12 33.83 32.10 30.80 30.05 30.12 28.57 27.93 nan   
2020-04-14    nan 37.88 32.98 32.05 31.02 29.93 29.35 29.88 28.35 27.65 nan   
2020-04-15    nan 42.51 36.58 34.58 33.08 31.57 30.85 31.32 29.60 28.55 nan   
2020-04-16    nan 36.52 34.42 33.08 31.85 31.12 31.73 29.90 28.77   nan nan   
2020-04-17    nan 34.58 33.08 31.93 30.88 30.40 31.25 29.55 28.50   nan nan   
2020-04-20    nan 39.08 36.23 34.42 32.88 32.25 33.02 31.20 29.70 29.52 nan   
2020-04-21    nan 41.77 38.67 36.42 34.42 33.58 33.98 31.82 30.62 30.00 nan   
2020-04-22    nan 39.62 37.38 35.42 33.83 33.23 33.67 31.57 30.45 30.23 nan   
2020-04-23    nan 39.73 37.73 35.67 33.98 33.17 33.73 31.60 30.32 30.23 nan   
2020-04-24    nan 36.77 35.58 34.17 32.88 32.02 32.48 30.23 29.00 30.18 nan   
2020-04-27    nan 33.88 33.48 32.58 31.48 30.77 31.38 29.27 28.07 30.18 nan   
2020-04-28    nan 34.17 33.83 32.77 31.77 31.07 31.68 29.45 28.10 30.20 nan   
2020-04-29    nan 31.93 31.77 31.12 30.57 30.02 30.77 28.55 27.38 31.20 nan   
2020-04-30    nan 33.98 33.23 32.08 31.32 30.77 31.50 29.32 28.02 28.25 nan   
2020-05-01    nan 37.35 35.98 34.23 33.38 33.02 33.62 31.27 30.00 30.25 nan   
2020-05-04    nan 35.67 34.98 33.73 33.08 32.70 33.35 31.07 29.90 30.05 nan   
2020-05-05    nan 34.02 33.77 32.83 32.12 31.98 32.65 30.40 29.27 30.02 nan   
2020-05-06    nan 34.77 34.58 33.58 32.67 32.42 33.12 30.82 29.57 28.57 nan   
2020-05-07    nan 32.12 32.62 32.08 31.38 31.07 31.93 29.77 28.68 28.73 nan   
2020-05-08    nan 29.32 30.73 30.57 30.15 30.07 31.07 28.95 27.85 27.90 nan   
2020-05-11    nan 27.82 29.02 29.43 29.27 29.38 30.55 28.43 27.45 26.85 nan   
2020-05-12    nan 33.08 32.92 32.33 31.70 31.65 32.73 30.48 29.30 27.18 nan   
2020-05-13    nan 35.08 34.92 33.83 32.67 32.60 33.77 31.43 30.10 29.65 nan   
2020-05-14    nan 32.38 32.92 32.62 32.05 32.12 33.15 30.82 29.65 30.75 nan   
2020-05-15    nan 32.12 32.73 32.58 32.08 32.38 33.33 31.30 30.05 29.12 nan   
2020-05-18    nan 28.93 30.10 30.40 30.23 30.52 31.57 29.62 28.62 28.88 nan   
2020-05-19    nan 30.75 32.08 31.77 31.27 31.57 32.62 30.75 29.60 29.85 nan   
2020-05-20    nan 28.11 29.77 30.57 30.32 30.57 31.77 29.93 28.93 28.93 nan   
2020-05-21    nan 31.02 31.62 31.23 31.48 32.62 30.62 29.52 29.55   nan nan   
2020-05-22    nan 30.38 31.07 30.80 31.10 32.35 30.38 29.18 29.15   nan nan   
2020-05-26    nan 29.62 30.48 30.18 30.43 31.98 30.00 28.77 28.75 28.52 nan   
2020-05-27    nan 29.07 30.07 29.82 30.12 31.68 29.73 28.43 28.45 28.10 nan   
2020-05-28    nan 30.32 31.12 30.68 30.77 32.33 30.23 28.88 28.75 27.90 nan   
2020-05-29    nan 28.82 30.18 30.02 30.23 31.93 29.82 28.35 28.30 28.50 nan   
2020-06-01    nan 29.52 30.77 30.48 30.52 32.17 30.07 28.57 28.52 28.52 nan   
2020-06-02    nan 28.23 30.02 29.88 30.02 31.88 29.82 28.32 28.25 28.20 nan   
2020-06-03    nan 27.00 29.07 29.02 29.32 31.12 29.10 27.57 27.57 27.95 nan   
2020-06-04    nan 26.73 28.73 28.68 29.07 30.98 28.95 27.40 27.38 27.25 nan   
2020-06-05    nan 25.32 27.18 27.23 27.68 29.68 27.68 26.32 26.45 26.55 nan   
2020-06-08    nan 26.02 27.93 27.93 28.32 30.38 28.23 26.77 26.73 26.35 nan   
2020-06-09    nan 27.62 29.57 29.10 29.27 31.07 29.07 27.68 2

In [192]:
del df06AP2['Settle',0]
del df06AP2['Settle',10]
del df06AP2['Settle',11]
del df06AP2['Settle',12]

In [193]:
del df06AP2['diffSettleClose',0]
del df06AP2['diffSettleClose',10]
del df06AP2['diffSettleClose',11]
del df06AP2['diffSettleClose',12]

In [194]:
del df06AP2['rangeDayHL',0]
del df06AP2['rangeDayHL',10]
del df06AP2['rangeDayHL',11]
del df06AP2['rangeDayHL',12]

In [195]:
del df06AP2['rangeDayOC',0]
del df06AP2['rangeDayOC',10]
del df06AP2['rangeDayOC',11]
del df06AP2['rangeDayOC',12]

In [196]:
df06AP2.tail(200)

Settle                                                  \
vxTerm          1     2     3     4     5     6     7     8     9   
tradeDate                                                           
2019-11-06  14.78 16.32 17.57 18.23 18.38 18.57 18.62 18.70 18.80   
2019-11-07  14.62 16.18 17.43 18.12 18.27 18.52 18.57 18.70 18.75   
2019-11-08  14.28 16.02 17.32 18.07 18.23 18.50 18.57 18.70 18.85   
2019-11-11  14.07 15.82 17.12 17.88 18.07 18.38 18.48 18.60 18.70   
2019-11-12  13.88 15.82 17.18 17.93 18.18 18.43 18.52 18.62 18.77   
2019-11-13  13.97 15.82 17.18 17.93 18.12 18.43 18.52 18.62 18.73   
2019-11-14  13.72 15.62 16.98 17.77 17.98 18.32 18.43 18.55 18.65   
2019-11-15  12.97 15.12 16.57 17.52 17.82 18.12 18.25 18.40 18.50   
2019-11-18  12.68 15.03 16.48 17.43 17.73 18.07 18.23 18.35 18.45   
2019-11-19  13.07 15.22 16.57 17.52 17.88 18.23 18.38 18.48 18.55   
2019-11-20  13.01 15.18 16.82 17.77 18.07 18.43 18.57 18.68 18.80   
2019-11-21  15.32 17.02 17.98 18.27 18.62 18.77 18.88 18.98   nan   
2019-11-22  14.78 16.73 17.77 18.07 18.43 18.62 18.75 18.90   nan   
2019-11-25  14.07 16.23 17.43 17.73 18.07 18.32 18.43 18.62 18.75   
2019-11-26  13.88 16.07 17.27 17.57 17.98 18.18 18.32 18.52 18.55   
2019-11-27  13.82 16.12 17.32 17.57 18.02 18.23 18.32 18.55 18.55   
2019-11-29  14.12 16.38 17.68 17.88 18.23 18.38 18.50 18.65 18.62   
2019-12-02  15.22 17.07 18.23 18.27 18.73 18.82 18.93 19.05 19.05   
2019-12-03  16.23 17.57 18.57 18.52 18.88 19.02 19.12 19.32 19.38   
2019-12-04  15.32 16.98 18.12 18.23 18.62 18.80 18.88 19.12 19.18   
2019-12-05  15.03 16.77 17.93 18.07 18.52 18.68 18.77 19.00 19.12   
2019-12-06  14.43 16.38 17.52 17.68 18.23 18.43 18.52 18.80 18.85   
2019-12-09  15.62 17.12 18.07 18.07 18.57 18.68 18.77 18.98 19.07   
2019-12-10  15.28 16.88 17.93 18.02 18.52 18.57 18.68 18.93 19.05   
2019-12-11  14.78 16.57 17.62 17.82 18.32 18.38 18.52 18.77 18.85   
2019-12-12  13.62 15.93 17.12 17.38 17.93 18.02 18.20 18.48 18.60   
2019-12-13  12.57 15.18 16.62 16.98 17.57 17.62 17.82 18.12 18.25   
2019-12-16  12.22 14.82 16.43 16.77 17.32 17.43 17.62 17.98 18.07   
2019-12-17  12.47 14.88 16.43 16.88 17.48 17.62 17.93 18.15 18.23   
2019-12-18  11.99 14.82 16.43 16.82 17.43 17.57 17.88 18.12 18.25   
2019-12-19  14.62 16.27 16.62 17.23 17.38 17.73 17.98 18.12 18.48   
2019-12-20  14.78 16.43 16.73 17.27 17.43 17.70 17.98 18.15 18.43   
2019-12-23  14.78 16.57 16.77 17.27 17.43 17.73 18.02 18.12 18.45   
2019-12-24  14.53 16.48 16.73 17.23 17.38 17.73 18.02 18.15 18.48   
2019-12-26  14.53 16.57 16.77 17.23 17.38 17.68 17.98 18.05 18.40   
2019-12-27  15.12 17.02 17.18 17.62 17.77 18.02 18.23 18.27 18.55   
2019-12-30  15.53 17.38 17.52 17.90 18.02 18.32 18.48 18.52 18.65   
2019-12-31  14.62 16.62 16.93 17.43 17.57 17.88 18.12 18.18 18.50   
2020-01-02  14.07 16.12 16.52 17.02 17.18 17.48 17.73 17.82 18.23   
2020-01-03  15.03 16.73 16.88 17.32 17.48 17.77 18.02 18.07 18.38   
2020-01-06  14.88 16.68 16.88 17.27 17.43 17.73 17.98 18.07 18.30   
2020-01-07  14.68 16.52 16.82 17.23 17.38 17.62 17.82 17.90 18.25   
2020-01-08  14.18 16.12 16.43 16.93 17.07 17.38 17.62 17.75 18.10   
2020-01-09  13.62 15.78 16.12 16.68 16.88 17.18 17.43 17.52 17.98   
2020-01-10  13.57 15.72 16.07 16.57 16.82 17.12 17.38 17.52 17.95   
2020-01-13  13.18 15.38 15.78 16.38 16.62 16.98 17.23 17.38 17.70   
2020-01-14  13.07 15.22 15.68 16.30 16.57 16.98 17.25 17.38 17.68   
2020-01-15  12.93 15.18 15.57 16.18 16.43 16.82 17.12 17.27 17.60   
2020-01-16  12.57 14.93 15.38 15.97 16.23 16.62 16.98 17.12 17.48   
2020-01-17  12.53 14.93 15.38 16.02 16.32 16.73 17.02 17.18 17.48   
2020-01-21  12.93 15.07 15.53 16.02 16.25 16.68 17.00 17.18 17.50   
2020-01-22  12.62 15.18 15.72 16.27 16.52 16.88 17.12 17.32 17.57   
2020-01-23  15.03 15.68 16.27 16.48 16.82 17.07 17.23 17.50 19.70   
2020-01-24  16.02 16.23 16.68 16.73 17.12 17.32 17.48 17.73 19.93   
2020-01-27  17.77 17.32 17.52 17.43 17.77 17.98 18.07 18.12 20.12   
2020-01-2

In [197]:
df06AP2['SettleArrayMax'] = df06AP2.Settle[:-1:].max(axis=1)
df06AP2['SettleArrayMin'] = df06AP2.Settle[:-1:].min(axis=1)
df06AP2['diffMaxMin'] = df06AP2.SettleArrayMax - df06AP2.SettleArrayMin
df06AP2['sig8over9'] = np.where((df06AP2['Settle',8] > df06AP2['Settle',9]),1,0)
df06AP2['sig1over2'] = np.where((df06AP2['Settle',1] > df06AP2['Settle',2]),1,0)
df06AP2.tail(200)

Settle                                                  \
vxTerm          1     2     3     4     5     6     7     8     9   
tradeDate                                                           
2019-11-06  14.78 16.32 17.57 18.23 18.38 18.57 18.62 18.70 18.80   
2019-11-07  14.62 16.18 17.43 18.12 18.27 18.52 18.57 18.70 18.75   
2019-11-08  14.28 16.02 17.32 18.07 18.23 18.50 18.57 18.70 18.85   
2019-11-11  14.07 15.82 17.12 17.88 18.07 18.38 18.48 18.60 18.70   
2019-11-12  13.88 15.82 17.18 17.93 18.18 18.43 18.52 18.62 18.77   
2019-11-13  13.97 15.82 17.18 17.93 18.12 18.43 18.52 18.62 18.73   
2019-11-14  13.72 15.62 16.98 17.77 17.98 18.32 18.43 18.55 18.65   
2019-11-15  12.97 15.12 16.57 17.52 17.82 18.12 18.25 18.40 18.50   
2019-11-18  12.68 15.03 16.48 17.43 17.73 18.07 18.23 18.35 18.45   
2019-11-19  13.07 15.22 16.57 17.52 17.88 18.23 18.38 18.48 18.55   
2019-11-20  13.01 15.18 16.82 17.77 18.07 18.43 18.57 18.68 18.80   
2019-11-21  15.32 17.02 17.98 18.27 18.62 18.77 18.88 18.98   nan   
2019-11-22  14.78 16.73 17.77 18.07 18.43 18.62 18.75 18.90   nan   
2019-11-25  14.07 16.23 17.43 17.73 18.07 18.32 18.43 18.62 18.75   
2019-11-26  13.88 16.07 17.27 17.57 17.98 18.18 18.32 18.52 18.55   
2019-11-27  13.82 16.12 17.32 17.57 18.02 18.23 18.32 18.55 18.55   
2019-11-29  14.12 16.38 17.68 17.88 18.23 18.38 18.50 18.65 18.62   
2019-12-02  15.22 17.07 18.23 18.27 18.73 18.82 18.93 19.05 19.05   
2019-12-03  16.23 17.57 18.57 18.52 18.88 19.02 19.12 19.32 19.38   
2019-12-04  15.32 16.98 18.12 18.23 18.62 18.80 18.88 19.12 19.18   
2019-12-05  15.03 16.77 17.93 18.07 18.52 18.68 18.77 19.00 19.12   
2019-12-06  14.43 16.38 17.52 17.68 18.23 18.43 18.52 18.80 18.85   
2019-12-09  15.62 17.12 18.07 18.07 18.57 18.68 18.77 18.98 19.07   
2019-12-10  15.28 16.88 17.93 18.02 18.52 18.57 18.68 18.93 19.05   
2019-12-11  14.78 16.57 17.62 17.82 18.32 18.38 18.52 18.77 18.85   
2019-12-12  13.62 15.93 17.12 17.38 17.93 18.02 18.20 18.48 18.60   
2019-12-13  12.57 15.18 16.62 16.98 17.57 17.62 17.82 18.12 18.25   
2019-12-16  12.22 14.82 16.43 16.77 17.32 17.43 17.62 17.98 18.07   
2019-12-17  12.47 14.88 16.43 16.88 17.48 17.62 17.93 18.15 18.23   
2019-12-18  11.99 14.82 16.43 16.82 17.43 17.57 17.88 18.12 18.25   
2019-12-19  14.62 16.27 16.62 17.23 17.38 17.73 17.98 18.12 18.48   
2019-12-20  14.78 16.43 16.73 17.27 17.43 17.70 17.98 18.15 18.43   
2019-12-23  14.78 16.57 16.77 17.27 17.43 17.73 18.02 18.12 18.45   
2019-12-24  14.53 16.48 16.73 17.23 17.38 17.73 18.02 18.15 18.48   
2019-12-26  14.53 16.57 16.77 17.23 17.38 17.68 17.98 18.05 18.40   
2019-12-27  15.12 17.02 17.18 17.62 17.77 18.02 18.23 18.27 18.55   
2019-12-30  15.53 17.38 17.52 17.90 18.02 18.32 18.48 18.52 18.65   
2019-12-31  14.62 16.62 16.93 17.43 17.57 17.88 18.12 18.18 18.50   
2020-01-02  14.07 16.12 16.52 17.02 17.18 17.48 17.73 17.82 18.23   
2020-01-03  15.03 16.73 16.88 17.32 17.48 17.77 18.02 18.07 18.38   
2020-01-06  14.88 16.68 16.88 17.27 17.43 17.73 17.98 18.07 18.30   
2020-01-07  14.68 16.52 16.82 17.23 17.38 17.62 17.82 17.90 18.25   
2020-01-08  14.18 16.12 16.43 16.93 17.07 17.38 17.62 17.75 18.10   
2020-01-09  13.62 15.78 16.12 16.68 16.88 17.18 17.43 17.52 17.98   
2020-01-10  13.57 15.72 16.07 16.57 16.82 17.12 17.38 17.52 17.95   
2020-01-13  13.18 15.38 15.78 16.38 16.62 16.98 17.23 17.38 17.70   
2020-01-14  13.07 15.22 15.68 16.30 16.57 16.98 17.25 17.38 17.68   
2020-01-15  12.93 15.18 15.57 16.18 16.43 16.82 17.12 17.27 17.60   
2020-01-16  12.57 14.93 15.38 15.97 16.23 16.62 16.98 17.12 17.48   
2020-01-17  12.53 14.93 15.38 16.02 16.32 16.73 17.02 17.18 17.48   
2020-01-21  12.93 15.07 15.53 16.02 16.25 16.68 17.00 17.18 17.50   
2020-01-22  12.62 15.18 15.72 16.27 16.52 16.88 17.12 17.32 17.57   
2020-01-23  15.03 15.68 16.27 16.48 16.82 17.07 17.23 17.50 19.70   
2020-01-24  16.02 16.23 16.68 16.73 17.12 17.32 17.48 17.73 19.93   
2020-01-27  17.77 17.32 17.52 17.43 17.77 17.98 18.07 18.12 20.12   
2020-01-2

In [198]:
df06AP2.diffMaxMin.min()

0.0

In [199]:
df06AP2.to_csv('df-6-P1F.csv')